In [1]:
import os
import keyring
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Model Component

In [2]:
from openai import OpenAI

client = OpenAI()
client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {
            "role": "user",
            "content": "2022년 월드컵 4강 국가 알려줘."
        }
    ]
)

ChatCompletion(id='chatcmpl-AX0OijLOSPJ22yAaAM8nLqrVbPk5Q', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='죄송합니다. 현재로서는 2022년 월드컵 4강 국가를 알려 드릴 수 없습니다. 현재 시점에서는 아직 월드컵이 진행 중이거나 예정되지 않았기 때문입니다. 이 질문을 다시 한 번 월드컵이 진행 중인 시기에 물어 보시면 더 나은 대답을 드릴 수 있을 것입니다. 감사합니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1732429716, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=137, prompt_tokens=32, total_tokens=169, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model_name='gpt-4o-mini'
)
chat.invoke("안녕! 너를 소개해줄래?")

AIMessage(content='안녕하세요! 저는 인공지능 언어 모델인 ChatGPT입니다. 다양한 주제에 대해 대화하고 질문에 답변하며 정보 제공을 도와드릴 수 있습니다. 문학, 과학, 기술, 일상적인 질문 등 여러 분야에 걸쳐 여러분과 소통할 수 있습니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 17, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-53f42f3c-f208-4e01-b189-875293b4cb10-0', usage_metadata={'input_tokens': 17, 'output_tokens': 72, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# PromptTemplate

In [4]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template(
        """ 
        너는 요리사아. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,
        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.
        <재료>
        {재료}
        """
    )
)

prompt

PromptTemplate(input_variables=['개수', '재료'], input_types={}, partial_variables={}, template=' \n        너는 요리사아. 내가 가진 재료들을 갖고 만들 수 있는 요리를 {개수}추천하고,\n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        {재료}\n        ')

In [ ]:
# prompt template에 input_variables를 채울 경우
prompt.invoke({"개수": 6, "재료": "사과, 잼"})

StringPromptValue(text=' \n        너는 요리사아. 내가 가진 재료들을 갖고 만들 수 있는 요리를 6추천하고,\n        그 요리의 레시피를 제시해줘. 내가 가진 재료는 아래와 같아.\n        <재료>\n        사과, 잼\n        ')

In [8]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        #SystemMessage : 유용한 챗봇이라는 역할과 이름 부여
        ("system", "You are a helpful AI bot. Your name is {name}."),
        #HumanMessage와 AIMessage : 서로 안부를 묻고 답하는 대화 히스토리 주입
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        #HumanMessage로 사용자가 입력한 프롬프트 전달
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name='Bob', user_input="What is your name?")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


# Building Chain with LCEL

In [ ]:
from langchain_core.output_parsers import StrOutputParser       # llm의 응답값 중 metadata는 제외하고 str 정보만 parsing
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# set prompt template
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

# call the LLMs
model = ChatOpenAI(model='gpt-4o-mini')

# connect prompt template, llm output parser with LCEL
chain = prompt | model | StrOutputParser()

# execute the chain by calling invoke
chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down? \n\nBecause it drove over a rocky road! 🍦'

In [10]:
# declare Chain
model = ChatOpenAI(model='gpt-4o')
prompt = ChatPromptTemplate.from_template('tell me a joke about {topic}')
chain = prompt | model

# add streaming with Chain's stream() function
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why did the bear sit in front of the computer?

Because it wanted to keep an eye on its favorite "websites"!

# OutputParser

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# ChatPromptTemplate에 SystemMessage로 LLM의 역할과 출력 형식 지정
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "너는 영화 전문가 AI야. 사용자가 원하는 장르의 영화를 리스트 형태로 추천해줘."
                'ex) Query: SF영화 3개 추천해줘 / 답변: ["인터스텔라", "스페이스오디세이", "혹성탈출"]'
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

model = ChatOpenAI(model='gpt-4o-mini')
chain = chat_template | model
chain.invoke("액션")

AIMessage(content='["다크 나이트", "존 윅", "매드맥스: 분노의 도로"]', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 72, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-975c3890-73d9-423d-8a15-d8305cd32f0a-0', usage_metadata={'input_tokens': 72, 'output_tokens': 23, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

# CSV parser
output_parser = CommaSeparatedListOutputParser()

# load prompt for the CSV parser
format_instuctions = output_parser.get_format_instructions()
format_instuctions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [14]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

# CSV parser
output_parser = CommaSeparatedListOutputParser()

# load prompt for the CSV parser
format_instructions = output_parser.get_format_instructions()

# insert formatted instructions to the partial_variables of the prompt template
prompt = PromptTemplate(
    template="List {subject}. answer in Korean \n{format_instructions}",
    input_variables=['subject'],
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(model='gpt-4o-mini')

# Chaining PromptTemplate - Model - OutputParser
chain = prompt | model | output_parser
chain.invoke({"subject": "어린이 영화"})

['토이 스토리',
 '겨울왕국',
 '주토피아',
 '슈렉',
 '모아나',
 '코코',
 '벅스 라이프',
 '미니언즈',
 '인사이드 아웃',
 '주먹왕 랄프']

In [16]:
from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# define data structure
class Country(BaseModel):
    contient: str = Field(description="사용자가 물어본 나라가 속한 대륙")
    popluation: str = Field(description="사용자가 물어본 나라의 인구(int 형식)")
    
# JsonOutputParser를 설정하고 프롬프트 템플릿에 format_instructions을 삽입한다.
parser = JsonOutputParser(pydantic_object=Country)

prompt = PromptTemplate(
    template="Answer the user query. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

country_query = "아르헨티나는 어떤 나라야?"
chain.invoke({"query": country_query})

{'contient': '남아메리카', 'popluation': '45195777'}

# LCEL의 Runnable 객체에 대해 알아보기

## RunnablePassthrough 알아보기

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 주어진 텍스트 그대로 통과
RunnablePassthrough().invoke("안녕하세요.")

'안녕하세요.'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""다음 한글 문장을 프랑스어로 번역해줘 {sentence}
French sentence: (print from here)""")

model = ChatOpenAI(model='gpt-4o-mini')

output_parser = StrOutputParser()

runnable_chain = {"sentence": RunnablePassthrough()} | prompt | model | output_parser
runnable_chain.invoke({"sentence": "그녀는 매일 아침 책을 읽습니다."})

'Elle lit un livre chaque matin.'

In [ ]:
# assign 함수를 이용하면 변수를 하나 추가 가능
# num으로 들어온 숫자에 3을 곱해 mult에 저장
(RunnablePassthrough.assign(mult=lambda x: x["num"]*3)).invoke({"num": 3})

{'num': 3, 'mult': 9}

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# 병렬로 실행
# extra(RunnablePassthrough, assign), modified 두개 동시 실행
runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x['num']*3),
    modified=lambda x: x['num'] + 1,
)

runnable.invoke({"num": 1})

{'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## RunnableLambda 알아보기

In [22]:
def add_smile(x):
    return x + ":)"

In [ ]:
from langchain_core.runnables import RunnableLambda

# Runnable 객체 변환
add_smile = RunnableLambda(add_smile)

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt_str = "{topic}의 역사에 대해 세문장으로 설명해주세요."
prompt = ChatPromptTemplate.from_template(prompt_str)

model = ChatOpenAI(model_name='gpt-4o-mini')

output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [27]:
from langchain_core.runnables import RunnableLambda

def add_thanks(x):
    return x + " 들어주셔서 감사합니다 :)"

add_thanks = RunnableLambda(add_thanks)

In [28]:
chain = prompt | model | output_parser | add_thanks
chain.invoke("반도체")

'반도체의 역사는 19세기 중반으로 거슬러 올라가며, 초기에는 주로 연구와 실험적인 형태로 존재했습니다. 1947년 벨 연구소의 과학자들이 트랜지스터를 발명하면서 반도체 기술은 혁신적인 발전을 이루었고, 이를 통해 전자기기와 컴퓨터의 발전이 가속화되었습니다. 이후 반도체는 현대 전자 산업의 핵심 요소로 자리잡으며, 정보통신, 자동차, 의료 등 다양한 분야에서 필수적인 역할을 하고 있습니다. 들어주셔서 감사합니다 :)'

## RunnableParallel 알아보기

In [29]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

In [30]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=add_thanks,
)

runnable.invoke("안녕하세요.")

{'passed': '안녕하세요.', 'modified': '안녕하세요. 들어주셔서 감사합니다 :)'}

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(model='gpt-4o-mini', max_tokens=128, temperature=0)

history_prompt = ChatPromptTemplate.from_template("{topic}가 무엇의 약자인지 알려주세요.")
celeb_prompt = ChatPromptTemplate.from_template("{topic} 분야의 유명인사 3명의 이름만 알려주세요.")

output_parser = StrOutputParser()

history_chain = history_prompt | model | output_parser
celeb_chain = celeb_prompt | model | output_parser

map_chain = RunnableParallel(history_chain=history_chain, celeb=celeb_chain)

result = map_chain.invoke({"topic": "AI"})
result

{'history_chain': 'AI는 "Artificial Intelligence"의 약자로, 한국어로는 "인공지능"이라고 합니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결, 언어 이해 등의 작업을 수행할 수 있도록 하는 기술을 의미합니다.',
 'celeb': '1. 앤드류 응 (Andrew Ng)\n2. 제프리 힌튼 (Geoffrey Hinton)\n3. 얀 르쿤 (Yann LeCun)'}